# 🌋 Vesuvius Challenge 📜 Surface Detection

### Official Formula:

    Score = 0.30 × TopoScore + 0.35 × SurfaceDice@τ + 0.35 × VOI_score

### File Format:
- Input: 3D TIF files (320×320×320 uint8)
- Labels: 0=background, 1=surface, 2=unlabeled (ignored)

### Usage:
    
    gt = load_volume("path/to/gt.tif")
    pred = load_volume("path/to/pred.tif")
    result = compute_score(pred, gt)
    print(f"Score: {result['score']}")

### Details

Competition: https://www.kaggle.com/competitions/vesuvius-challenge-surface-detection

Visualisation at: https://www.kaggle.com/code/jirkaborovec/surface-eda-interactive-img-mask-3d-view

Co-authored by: claude.ai @ Opus 4.5

In [ ]:
!pip install -q imagecodecs

# ⚙️ CONSTANTS

**from competition specification**

In [ ]:
import time
import imagecodecs
import tifffile
import numpy as np
from scipy import ndimage
from scipy.ndimage import distance_transform_edt, label as cc3d_label
from skimage.measure import euler_number
from typing import Tuple, Dict, Union, List
from pathlib import Path

TAU = 2.0                    # Surface Dice tolerance
ALPHA_VOI = 0.3              # VOI conversion parameter
BETTI_WEIGHTS = {0: 0.34, 1: 0.33, 2: 0.33}  # TopoScore weights
W_TOPO = 0.30                # Final score weight for TopoScore
W_SURFACE_DICE = 0.35        # Final score weight for SurfaceDice
W_VOI = 0.35                 # Final score weight for VOI

LABEL_BACKGROUND = 0
LABEL_SURFACE = 1
LABEL_UNLABELED = 2

# I/O Functions & PreProcessing

In [ ]:
def load_volume(path: Union[str, Path]) -> np.ndarray:
    """Load 3D volume from TIF file."""
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"Volume not found: {path}")
    volume = tifffile.imread(str(path))
    if volume.ndim != 3:
        raise ValueError(f"Expected 3D volume, got shape {volume.shape}")
    return volume.astype(np.uint8)


def save_volume(
    volume: np.ndarray, path: Union[str, Path], compress: bool = True
):
    """Save 3D volume to TIF file."""
    compression = 'lzw' if compress else None
    tifffile.imwrite(str(path), volume.astype(np.uint8), compression=compression)


def preprocess(
    pred: np.ndarray, gt: np.ndarray, 
    ignore_label: int = LABEL_UNLABELED
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Binarize volumes and create valid mask (excluding ignore_label)."""
    valid_mask = (gt != ignore_label)
    pred_bin = ((pred == LABEL_SURFACE) & valid_mask).astype(np.uint8)
    gt_bin = ((gt == LABEL_SURFACE) & valid_mask).astype(np.uint8)
    return pred_bin, gt_bin, valid_mask

In [ ]:
def get_components(
    volume: np.ndarray
) -> Tuple[np.ndarray, int, Dict[int, int]]:
    """Get connected components with sizes."""
    struct_26 = ndimage.generate_binary_structure(3, 3)
    labels, n = cc3d_label(volume.astype(bool), structure=struct_26)
    sizes = {i: int(np.sum(labels == i)) for i in range(1, n + 1)}
    return labels, n, sizes


def remove_component(
    volume: np.ndarray, labels: np.ndarray, comp_id: int
) -> np.ndarray:
    """Remove specific component."""
    result = volume.copy()
    result[labels == comp_id] = LABEL_BACKGROUND
    return result


def remove_components(
    volume: np.ndarray, labels: np.ndarray, comp_ids: List[int]
) -> np.ndarray:
    """Remove multiple components."""
    result = volume.copy()
    for cid in comp_ids:
        result[labels == cid] = LABEL_BACKGROUND
    return result


def add_hole_in_component(
    volume: np.ndarray,
    labels: np.ndarray,
    comp_id: int,
    hole_size: int = 25
) -> Tuple[np.ndarray, int]:
    """
    Add a rectangular hole in the center of a specific component.
    
    Creates a topology break (adds β₁ tunnel) without affecting other components.
    
    Args:
        volume: Label volume
        labels: Connected component labels
        comp_id: Which component to add hole to
        hole_size: Half-size of hole in Y and Z dimensions
        
    Returns:
        (modified_volume, voxels_removed)
    """
    comp_mask = (labels == comp_id)
    coords = np.where(comp_mask)
    
    if len(coords[0]) == 0:
        return volume.copy(), 0
    
    # Find center of component
    z_center = int(np.mean(coords[0]))
    y_center = int(np.mean(coords[1]))
    
    # Create rectangular hole region
    D, H, W = volume.shape
    z_grid = np.arange(D)[:, None, None]
    y_grid = np.arange(H)[None, :, None]
    
    hole_region = (
        (z_grid >= z_center - hole_size) & 
        (z_grid <= z_center + hole_size) &
        (y_grid >= y_center - hole_size) & 
        (y_grid <= y_center + hole_size)
    )
    
    # Only remove voxels in BOTH hole region AND this component
    hole_mask = hole_region & comp_mask
    
    result = volume.copy()
    result[hole_mask] = LABEL_BACKGROUND
    
    return result, int(np.sum(hole_mask))


def analyze_volume(volume: np.ndarray, name: str = "Volume") -> Dict:
    """Analyze label volume and return statistics."""
    surface = (volume == LABEL_SURFACE).astype(np.uint8)
    labels, n_comp, sizes = get_components(surface)
    betti = compute_betti_numbers(surface)
    
    return {
        'shape': volume.shape,
        'n_surface_voxels': int(np.sum(surface)),
        'n_components': n_comp,
        'betti': betti,
        'component_labels': labels,
        'component_sizes': sizes
    }


def add_hole_in_component(
    volume: np.ndarray,
    labels: np.ndarray,
    comp_id: int,
    hole_size: int = 25,
    label_bg: int = 0,
) -> Tuple[np.ndarray, int]:
    """Add a rectangular hole in the center of a specific component.
    
    Creates a topology break (adds β₁ tunnel) without affecting other components.
    
    Args:
        volume: Label volume
        labels: Connected component labels
        comp_id: Which component to add hole to
        hole_size: Half-size of hole in Y and Z dimensions
        
    Returns:
        (modified_volume, voxels_removed)
    """
    comp_mask = (labels == comp_id)
    coords = np.where(comp_mask)
    
    if len(coords[0]) == 0:
        return volume.copy(), 0
    
    # Find center of component
    z_center = int(np.mean(coords[0]))
    y_center = int(np.mean(coords[1]))
    
    # Create rectangular hole region
    D, H, W = volume.shape
    z_grid = np.arange(D)[:, None, None]
    y_grid = np.arange(H)[None, :, None]
    
    hole_region = (
        (z_grid >= z_center - hole_size) & 
        (z_grid <= z_center + hole_size) &
        (y_grid >= y_center - hole_size) & 
        (y_grid <= y_center + hole_size)
    )
    
    # Only remove voxels in BOTH hole region AND this component
    hole_mask = hole_region & comp_mask
    
    result = volume.copy()
    result[hole_mask] = label_bg
    
    return result, int(np.sum(hole_mask))

# 1 🎲 Surface Dice @ τ

In [ ]:
def extract_surface(volume: np.ndarray) -> np.ndarray:
    """Extract boundary voxels using 6-connectivity erosion."""
    if np.sum(volume) == 0:
        return np.zeros_like(volume, dtype=np.uint8)
    struct_6 = ndimage.generate_binary_structure(3, 1)
    eroded = ndimage.binary_erosion(volume.astype(bool), structure=struct_6)
    return volume.astype(np.uint8) - eroded.astype(np.uint8)


def compute_surface_dice(
    pred: np.ndarray, gt: np.ndarray, tau: float = TAU
) -> float:
    """
    Surface Dice @ τ: fraction of surface points within tolerance.
    
    Edge cases: both empty → 1.0, one empty → 0.0
    """
    pred_empty = np.sum(pred) == 0
    gt_empty = np.sum(gt) == 0
    
    if pred_empty and gt_empty:
        return 1.0
    if pred_empty or gt_empty:
        return 0.0
    
    pred_surface = extract_surface(pred)
    gt_surface = extract_surface(gt)
    
    n_pred = np.sum(pred_surface)
    n_gt = np.sum(gt_surface)
    
    if n_pred == 0 or n_gt == 0:
        return 0.0
    
    # Distance from pred surface to GT SURFACE (not volume!)
    dist_to_gt_surface = distance_transform_edt(~gt_surface.astype(bool))
    pred_matched = np.sum(dist_to_gt_surface[pred_surface > 0] <= tau)
    
    # Distance from GT surface to pred SURFACE (not volume!)
    dist_to_pred_surface = distance_transform_edt(~pred_surface.astype(bool))
    gt_matched = np.sum(dist_to_pred_surface[gt_surface > 0] <= tau)
    
    return float((pred_matched + gt_matched) / (n_pred + n_gt))

# 2 📦 VOI Score

In [ ]:
def compute_voi(
    pred_labels: np.ndarray, gt_labels: np.ndarray
) -> Tuple[float, float, float]:
    """Compute Variation of Information: (VOI_split, VOI_merge, VOI_total)."""
    n = pred_labels.size
    if n == 0:
        return 0.0, 0.0, 0.0
    
    max_pred = int(np.max(pred_labels)) + 1
    max_gt = int(np.max(gt_labels)) + 1
    
    contingency = np.zeros((max_pred, max_gt), dtype=np.float64)
    for p, g in zip(pred_labels.ravel(), gt_labels.ravel()):
        contingency[p, g] += 1
    
    p_ij = contingency / n
    p_i = np.sum(p_ij, axis=1)
    p_j = np.sum(p_ij, axis=0)
    
    voi_split = 0.0
    voi_merge = 0.0
    
    for i in range(max_pred):
        for j in range(max_gt):
            if p_ij[i, j] > 0:
                if p_i[i] > 0:
                    voi_split -= p_ij[i, j] * np.log2(p_ij[i, j] / p_i[i])
                if p_j[j] > 0:
                    voi_merge -= p_ij[i, j] * np.log2(p_ij[i, j] / p_j[j])
    
    return float(voi_split), float(voi_merge), float(voi_split + voi_merge)


def compute_voi_score(pred: np.ndarray, gt: np.ndarray) -> float:
    """VOI Score = 1 / (1 + α × VOI_total). Uses 26-connectivity."""
    if np.sum(pred) == 0 and np.sum(gt) == 0:
        return 1.0
    if np.sum(pred) == 0 or np.sum(gt) == 0:
        return 0.0
    
    struct_26 = ndimage.generate_binary_structure(3, 3)
    pred_labels, _ = cc3d_label(pred.astype(bool), structure=struct_26)
    gt_labels, _ = cc3d_label(gt.astype(bool), structure=struct_26)
    
    union_fg = (pred.astype(bool) | gt.astype(bool))
    _, _, voi_total = compute_voi(pred_labels[union_fg], gt_labels[union_fg])
    
    return float(1.0 / (1.0 + ALPHA_VOI * voi_total))

# 3 🏔️ TOPO Score

In [ ]:
def compute_betti_numbers(volume: np.ndarray) -> Tuple[int, int, int]:
    """Compute Betti numbers (β₀, β₁, β₂) for 3D binary volume."""
    if np.sum(volume) == 0:
        return (0, 0, 0)
    
    vol_bool = volume.astype(bool)
    struct_26 = ndimage.generate_binary_structure(3, 3)
    _, beta0 = cc3d_label(vol_bool, structure=struct_26)
    
    # Cavities (enclosed background components)
    struct_6 = ndimage.generate_binary_structure(3, 1)
    bg_labels, n_bg = cc3d_label(~vol_bool, structure=struct_6)
    
    boundary_labels = set()
    for face in [bg_labels[0,:,:], bg_labels[-1,:,:],
                 bg_labels[:,0,:], bg_labels[:,-1,:],
                 bg_labels[:,:,0], bg_labels[:,:,-1]]:
        boundary_labels.update(face.ravel())
    boundary_labels.discard(0)
    beta2 = n_bg - len(boundary_labels)
    
    # β₁ from Euler characteristic
    chi = euler_number(vol_bool, connectivity=3)
    beta1 = max(0, beta0 - chi + beta2)
    
    return (int(beta0), int(beta1), int(beta2))


def compute_betti_f1(pred_betti: int, gt_betti: int) -> float:
    """F1 score for Betti number matching."""
    if pred_betti == 0 and gt_betti == 0:
        return 1.0
    if pred_betti == 0 or gt_betti == 0:
        return 0.0
    matched = min(pred_betti, gt_betti)
    precision = matched / pred_betti
    recall = matched / gt_betti
    if precision + recall == 0:
        return 0.0
    return 2.0 * precision * recall / (precision + recall)


def compute_topo_score(
    pred: np.ndarray, gt: np.ndarray
) -> Tuple[float, Tuple, Tuple]:
    """TopoScore: weighted Betti F1. Returns (score, pred_betti, gt_betti)."""
    pred_betti = compute_betti_numbers(pred)
    gt_betti = compute_betti_numbers(gt)
    
    f1_scores = {}
    active_weights = {}
    
    for k in range(3):
        pb, gb = pred_betti[k], gt_betti[k]
        if pb > 0 or gb > 0:
            f1_scores[k] = compute_betti_f1(pb, gb)
            active_weights[k] = BETTI_WEIGHTS[k]
    
    if not active_weights:
        return 1.0, pred_betti, gt_betti
    
    total_weight = sum(active_weights.values())
    score = sum(f1_scores[k] * active_weights[k] for k in active_weights) / total_weight
    
    return float(score), pred_betti, gt_betti

# ⚖️ MAIN 🏆 Scoring function

In [ ]:
from functools import wraps

def time_log(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"[Timer] {func.__name__:20} | {end - start:.4f}s")
        return result
    return wrapper

In [ ]:
def compute_score(
    pred: np.ndarray,
    gt: np.ndarray,
    tau: float = TAU,
    ignore_label: int = LABEL_UNLABELED,
    verbose: bool = False
) -> Dict:
    """Compute official Vesuvius Surface Detection competition score.
    
    Score = 0.30 × TopoScore + 0.35 × SurfaceDice@τ + 0.35 × VOI_score
    """
    # 1. Preprocess
    t0 = time.perf_counter()
    pred_bin, gt_bin, _ = preprocess(pred, gt, ignore_label)
    t_pre = time.perf_counter() - t0

    # 2. Surface Dice
    t1 = time.perf_counter()
    surface_dice = compute_surface_dice(pred_bin, gt_bin, tau)
    t_dice = time.perf_counter() - t1

    # 3. VOI
    t2 = time.perf_counter()
    voi_score = compute_voi_score(pred_bin, gt_bin)
    t_voi = time.perf_counter() - t2

    # 4. Topo Score
    t3 = time.perf_counter()
    topo_score, pred_betti, gt_betti = compute_topo_score(pred_bin, gt_bin)
    t_topo = time.perf_counter() - t3
    
    score = W_TOPO * topo_score + W_SURFACE_DICE * surface_dice + W_VOI * voi_score
    
    result = {
        'score': round(score, 6),
        'surface_dice': round(surface_dice, 6),
        'voi_score': round(voi_score, 6),
        'topo_score': round(topo_score, 6),
        'pred_betti': pred_betti,
        'gt_betti': gt_betti
    }
    
    if verbose:
        print(f"\n--- Timing Breakdown ---")
        print(f"Preprocess:   {t_pre:.4f}s")
        print(f"Surface Dice: {t_dice:.4f}s")
        print(f"VOI Score:    {t_voi:.4f}s")
        print(f"Topo Score:   {t_topo:.4f}s")
        print(f"--- Particular Metrics ---")
        print(f"SurfaceDice@{tau}: {surface_dice:.6f}")
        print(f"VOI_score:        {voi_score:.6f}")
        print(f"TopoScore:        {topo_score:.6f}")
        print(f"  Pred β: {pred_betti}, GT β: {gt_betti}")
        print(f">>> SCORE:        {score:.6f}\n")
    
    return result


def score_from_files(
    pred_path: Union[str, Path],
    gt_path: Union[str, Path],
    tau: float = TAU,
    verbose: bool = True
) -> Dict:
    """Compute score directly from TIF file paths."""
    if verbose:
        print(f"Loading: {pred_path}")
    pred = load_volume(pred_path)
    
    if verbose:
        print(f"Loading: {gt_path}")
    gt = load_volume(gt_path)
    
    if pred.shape != gt.shape:
        raise ValueError(f"Shape mismatch: pred {pred.shape} vs gt {gt.shape}")
    
    if verbose:
        print(f"Shape: {gt.shape}")
        print("-" * 50)
    
    return compute_score(pred, gt, tau=tau, verbose=verbose)

# 🚀 Real 📽️ DEMO

In [ ]:
print("=" * 70)
print("VESUVIUS SURFACE DETECTION - KAGGLE COMPETITION METRIC")
print("=" * 70)
print()
print("Score = 0.30×TopoScore + 0.35×SurfaceDice@2.0 + 0.35×VOI_score")
print()

# # Score from files
# result = score_from_files(sys.argv[1], sys.argv[2], verbose=True)

# Analyze single file and run instance tests
gt_path = "/kaggle/input/vesuvius-challenge-surface-detection/train_labels/602831951.tif"
print(f"Loading: {gt_path}")
gt = load_volume(gt_path)

# Analyze
info = analyze_volume(gt)
print(f"\nVolume: {info['shape']}")
print(f"Surface voxels: {info['n_surface_voxels']:,}")
print(f"Components: {info['n_components']}")
print(f"Betti: β₀={info['betti'][0]}, β₁={info['betti'][1]}, β₂={info['betti'][2]}")

# Sort components by size
sorted_comps = sorted(info['component_sizes'].items(), key=lambda x: -x[1])
print(f"\nTop 5 components:")
for i, (cid, size) in enumerate(sorted_comps[:5]):
    pct = 100 * size / info['n_surface_voxels']
    print(f"  #{i+1} (ID {cid}): {size:,} voxels ({pct:.1f}%)")

# Run test cases
print("\n" + "=" * 70)
print("INSTANCE-BASED TEST CASES")
print("=" * 70)

labels = info['component_labels']
largest_id = sorted_comps[0][0]

tests = [
    ("Perfect Match", gt.copy()),
    ("Remove 1 Surface", remove_component(gt.copy(), labels, sorted_comps[0][0])),
    ("Remove 3 Surfaces", remove_components(gt.copy(), labels, [c[0] for c in sorted_comps[:3]])),
    # ("Remove Half", remove_components(gt.copy(), labels, [c[0] for c in sorted_comps[:len(sorted_comps)//2]])),
    ("Empty Prediction", np.zeros_like(gt)),
]
for hole in (25, 50, 100, 150):
    # Create hole in largest component
    pred_with_hole, voxels_removed = add_hole_in_component(gt.copy(), labels, largest_id, hole_size=hole)
    tests.append((f"Hole in Surface (−{voxels_removed:,} vox)", pred_with_hole)) 

results = []
for name, pred in tests:
    r = compute_score(pred, gt, verbose=True)
    results.append(f"{name:<35}" \
        f" {r['surface_dice']:>10.4f} {r['voi_score']:>10.4f} {r['topo_score']:>10.4f} {r['score']:>10.4f}")

print(f"\n{'Case':<35} {'SurfDice':>10} {'VOI':>10} {'Topo':>10} {'SCORE':>10}")
print("-" * 70)
for res in results:
    print(res)